In [1]:
!git clone https://github.com/thtrieu/darkflow



Cloning into 'darkflow'...


In [2]:
!pip install -e


Usage:   
  pip install [options] <requirement specifier> [package-index-options] ...
  pip install [options] -r <requirements file> [package-index-options] ...
  pip install [options] [-e] <vcs project url> ...
  pip install [options] [-e] <local project path> ...
  pip install [options] <archive url/path> ...

-e option requires 1 argument
